# Import

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [41]:
!pip install shap

In [42]:
import shap
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Load trainset

In [43]:
df = pd.read_csv('/content/drive/MyDrive/Dataset/insdn/binary/train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275111 entries, 0 to 275110
Data columns (total 77 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow Duration      275111 non-null  float64
 1   Tot Fwd Pkts       275111 non-null  float64
 2   Tot Bwd Pkts       275111 non-null  float64
 3   TotLen Fwd Pkts    275111 non-null  float64
 4   TotLen Bwd Pkts    275111 non-null  float64
 5   Fwd Pkt Len Max    275111 non-null  float64
 6   Fwd Pkt Len Min    275111 non-null  float64
 7   Fwd Pkt Len Mean   275111 non-null  float64
 8   Fwd Pkt Len Std    275111 non-null  float64
 9   Bwd Pkt Len Max    275111 non-null  float64
 10  Bwd Pkt Len Min    275111 non-null  float64
 11  Bwd Pkt Len Mean   275111 non-null  float64
 12  Bwd Pkt Len Std    275111 non-null  float64
 13  Flow Byts/s        275111 non-null  float64
 14  Flow Pkts/s        275111 non-null  float64
 15  Flow IAT Mean      275111 non-null  float64
 16  Fl

In [44]:
df = df.drop(columns=['Label'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275111 entries, 0 to 275110
Data columns (total 76 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow Duration      275111 non-null  float64
 1   Tot Fwd Pkts       275111 non-null  float64
 2   Tot Bwd Pkts       275111 non-null  float64
 3   TotLen Fwd Pkts    275111 non-null  float64
 4   TotLen Bwd Pkts    275111 non-null  float64
 5   Fwd Pkt Len Max    275111 non-null  float64
 6   Fwd Pkt Len Min    275111 non-null  float64
 7   Fwd Pkt Len Mean   275111 non-null  float64
 8   Fwd Pkt Len Std    275111 non-null  float64
 9   Bwd Pkt Len Max    275111 non-null  float64
 10  Bwd Pkt Len Min    275111 non-null  float64
 11  Bwd Pkt Len Mean   275111 non-null  float64
 12  Bwd Pkt Len Std    275111 non-null  float64
 13  Flow Byts/s        275111 non-null  float64
 14  Flow Pkts/s        275111 non-null  float64
 15  Flow IAT Mean      275111 non-null  float64
 16  Fl

# Load adv samples

In [45]:
feature_list = ['Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts',
       'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min',
       'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max',
       'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s',
       'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len',
       'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min',
       'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var',
       'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt',
       'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt',
       'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg',
       'Fwd Byts/b Avg', 'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg',
       'Bwd Byts/b Avg', 'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg',
       'Subflow Fwd Pkts', 'Subflow Fwd Byts', 'Subflow Bwd Pkts',
       'Subflow Bwd Byts', 'Init Fwd Win Byts', 'Init Bwd Win Byts',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
       'Idle Min']

In [46]:
pgd_path = '/content/drive/MyDrive/Dataset/insdn/adv/24_feature/pgd/pgd_mlp_bin_feature'
#pgd_label_path = '/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/adv/pgd/adv_pgd_target.csv'

In [47]:
pgd_df = pd.read_csv(pgd_path)

In [48]:
pgd_df.columns = feature_list
pgd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55151 entries, 0 to 55150
Data columns (total 76 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Flow Duration      55151 non-null  float64
 1   Tot Fwd Pkts       55151 non-null  float64
 2   Tot Bwd Pkts       55151 non-null  float64
 3   TotLen Fwd Pkts    55151 non-null  float64
 4   TotLen Bwd Pkts    55151 non-null  float64
 5   Fwd Pkt Len Max    55151 non-null  float64
 6   Fwd Pkt Len Min    55151 non-null  float64
 7   Fwd Pkt Len Mean   55151 non-null  float64
 8   Fwd Pkt Len Std    55151 non-null  float64
 9   Bwd Pkt Len Max    55151 non-null  float64
 10  Bwd Pkt Len Min    55151 non-null  float64
 11  Bwd Pkt Len Mean   55151 non-null  float64
 12  Bwd Pkt Len Std    55151 non-null  float64
 13  Flow Byts/s        55151 non-null  float64
 14  Flow Pkts/s        55151 non-null  float64
 15  Flow IAT Mean      55151 non-null  float64
 16  Flow IAT Std       551

In [ ]:
#pgd_label = pd.read_csv(pgd_label_path)
#pgd_label.info()

# Load DL-based IDS model

In [50]:
model_mlp = tf.keras.models.load_model('/content/drive/MyDrive/Dataset/insdn/insdn_mlp_bin.h5', compile=True)

# SHAP

In [51]:
explainer = shap.KernelExplainer(model_mlp, shap.sample(df, 100, random_state = 42))

In [52]:
pgd_1000 = pgd_df.head(1000)

In [53]:
shap_values = explainer.shap_values(pgd_1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [54]:
print(shap_values[0])

[[ 0.01482897 -0.00586572  0.         ...  0.02295556  0.00714035
   0.01116408]
 [ 0.          0.00557543  0.         ...  0.02118775  0.00593652
   0.0047586 ]
 [ 0.01687225  0.          0.00445177 ...  0.02799979 -0.00465082
  -0.02073587]
 ...
 [-0.00972969 -0.0016667   0.         ...  0.02772486 -0.05179911
   0.00861159]
 [ 0.          0.          0.00315754 ...  0.02522257  0.
   0.0044491 ]
 [ 0.         -0.01013362  0.01009805 ...  0.01486628 -0.00968522
   0.00845402]]


In [55]:
type(shap_values[0])

numpy.ndarray

In [56]:
data_df = pd.DataFrame(shap_values[0])

In [57]:
data_df.head()

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
0,0.014829,-0.005866,0.000000,-0.002392,-0.001471,0.000000,0.000000,-0.002094,0.014182,-0.001089,...,0.001928,0.0,0.044965,-0.015693,0.012960,0.056173,0.002285,0.022956,0.007140,0.011164
1,0.000000,0.005575,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005402,...,0.000000,0.0,0.012395,-0.066325,0.041687,0.028643,-0.019982,0.021188,0.005937,0.004759
2,0.016872,0.000000,0.004452,0.000000,0.000000,-0.004960,0.017292,0.006042,0.000000,0.007244,...,0.003034,0.0,-0.004542,-0.076643,0.007084,0.042095,-0.004078,0.028000,-0.004651,-0.020736
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.005602,0.0,0.022988,-0.035345,0.033660,0.055772,0.005243,0.015661,-0.020803,-0.016848
4,-0.068915,-0.004119,0.000000,0.000000,-0.005837,0.005555,0.000000,-0.010807,0.000000,0.019594,...,0.011318,0.0,0.031593,-0.276743,0.063275,0.076507,-0.160167,-0.002763,-0.140512,-0.197688


In [ ]:
data_df.to_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/InSDN/MLP_pgd/1000_samples_[0].csv', index=False)

In [58]:
data_df.columns = feature_list
data_df.head()

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0.014829,-0.005866,0.000000,-0.002392,-0.001471,0.000000,0.000000,-0.002094,0.014182,-0.001089,...,0.001928,0.0,0.044965,-0.015693,0.012960,0.056173,0.002285,0.022956,0.007140,0.011164
1,0.000000,0.005575,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005402,...,0.000000,0.0,0.012395,-0.066325,0.041687,0.028643,-0.019982,0.021188,0.005937,0.004759
2,0.016872,0.000000,0.004452,0.000000,0.000000,-0.004960,0.017292,0.006042,0.000000,0.007244,...,0.003034,0.0,-0.004542,-0.076643,0.007084,0.042095,-0.004078,0.028000,-0.004651,-0.020736
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.005602,0.0,0.022988,-0.035345,0.033660,0.055772,0.005243,0.015661,-0.020803,-0.016848
4,-0.068915,-0.004119,0.000000,0.000000,-0.005837,0.005555,0.000000,-0.010807,0.000000,0.019594,...,0.011318,0.0,0.031593,-0.276743,0.063275,0.076507,-0.160167,-0.002763,-0.140512,-0.197688


In [59]:
data_dict = {}
for idx in range(0, 1000):
  # Select the desired row and get the top 10 values's column names
  row_values = data_df.iloc[idx].nlargest(10).index.tolist()

  # Create a dictionary and add the list with filename as the key
  data_dict['Adv sample ' + str(idx)] = row_values

In [60]:
print(data_dict)

{'Adv sample 0': ['Active Min', 'Init Bwd Win Byts', 'Active Mean', 'Bwd IAT Min', 'Idle Std', 'Bwd IAT Std', 'Bwd IAT Mean', 'Down/Up Ratio', 'Flow Duration', 'Fwd Pkt Len Std'], 'Adv sample 1': ['SYN Flag Cnt', 'Init Bwd Win Byts', 'Bwd IAT Min', 'Active Max', 'Fwd IAT Min', 'Active Min', 'Bwd IAT Std', 'Idle Std', 'Bwd IAT Max', 'Active Mean'], 'Adv sample 2': ['SYN Flag Cnt', 'Init Bwd Win Byts', 'Active Min', 'Bwd IAT Mean', 'Fwd IAT Min', 'Idle Std', 'Fwd Byts/b Avg', 'Bwd IAT Std', 'Bwd IAT Min', 'Bwd IAT Max'], 'Adv sample 3': ['SYN Flag Cnt', 'Active Min', 'Init Bwd Win Byts', 'Fwd IAT Min', 'Bwd IAT Mean', 'Active Max', 'Bwd IAT Max', 'Active Mean', 'Idle Std', 'Bwd IAT Std'], 'Adv sample 4': ['ACK Flag Cnt', 'Bwd IAT Std', 'Active Min', 'Active Max', 'Init Bwd Win Byts', 'Fwd IAT Min', 'Active Mean', 'Flow IAT Mean', 'Fwd Byts/b Avg', 'Bwd Pkt Len Max'], 'Adv sample 5': ['Active Mean', 'Bwd IAT Max', 'Bwd IAT Mean', 'Init Bwd Win Byts', 'Fwd IAT Min', 'Idle Max', 'Fwd Pkt Le

In [61]:
len(shap_values[0])

1000

In [62]:
len(data_dict)

1000

# Detection phase

In [75]:
import os
import pandas as pd

In [76]:
whitelist_df = pd.read_csv('/content/drive/MyDrive/MLP_top24_InSDN.csv')
whitelist_df.head(24)

,Feature,Frequency
0,Init Bwd Win Byts,786
1,Idle Max,546
2,Bwd Pkt Len Max,531
3,Idle Mean,449
4,Tot Fwd Pkts,432
5,Idle Min,413
6,Bwd Pkt Len Std,399
7,Tot Bwd Pkts,395
8,Bwd Pkt Len Mean,382
9,TotLen Fwd Pkts,373


## Detection rate (top 51) = 991/1000

In [89]:
whitelist = whitelist_df.iloc[:, 0][:51].tolist()
print(whitelist)

['Init Bwd Win Byts', 'Idle Max', 'Bwd Pkt Len Max', 'Idle Mean', 'Tot Fwd Pkts', 'Idle Min', 'Bwd Pkt Len Std', 'Tot Bwd Pkts', 'Bwd Pkt Len Mean', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Bwd IAT Max', 'Bwd IAT Tot', 'Fwd IAT Max', 'Bwd Seg Size Avg', 'Pkt Len Min', 'SYN Flag Cnt', 'Flow Duration', 'Fwd Pkt Len Std', 'Fwd IAT Tot', 'Bwd IAT Std', 'Flow Pkts/s', 'Down/Up Ratio', 'Fwd IAT Std', 'Fwd Pkt Len Min', 'Fwd Pkt Len Max', 'Subflow Bwd Pkts', 'Bwd Pkt Len Min', 'Flow IAT Mean', 'Flow IAT Min', 'Fwd IAT Min', 'Bwd IAT Mean', 'Bwd IAT Min', 'Bwd Header Len', 'Subflow Bwd Byts', 'Subflow Fwd Byts', 'Fwd Header Len', 'Bwd PSH Flags', 'Idle Std', 'Fwd Pkts/s', 'Fwd Act Data Pkts', 'Pkt Len Max', 'Fwd IAT Mean', 'Active Max', 'Active Std', 'Pkt Len Std', 'Pkt Len Var', 'Subflow Fwd Pkts', 'Flow Byts/s', 'Pkt Size Avg', 'Active Mean']


In [90]:
alert = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist:
        alert.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert)))

Alert list length:991


## Detection rate (top 59) = 729/1000

In [85]:
whitelist2 = whitelist_df.iloc[:, 0][:60].tolist()
print(whitelist2)

['Init Bwd Win Byts', 'Idle Max', 'Bwd Pkt Len Max', 'Idle Mean', 'Tot Fwd Pkts', 'Idle Min', 'Bwd Pkt Len Std', 'Tot Bwd Pkts', 'Bwd Pkt Len Mean', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Bwd IAT Max', 'Bwd IAT Tot', 'Fwd IAT Max', 'Bwd Seg Size Avg', 'Pkt Len Min', 'SYN Flag Cnt', 'Flow Duration', 'Fwd Pkt Len Std', 'Fwd IAT Tot', 'Bwd IAT Std', 'Flow Pkts/s', 'Down/Up Ratio', 'Fwd IAT Std', 'Fwd Pkt Len Min', 'Fwd Pkt Len Max', 'Subflow Bwd Pkts', 'Bwd Pkt Len Min', 'Flow IAT Mean', 'Flow IAT Min', 'Fwd IAT Min', 'Bwd IAT Mean', 'Bwd IAT Min', 'Bwd Header Len', 'Subflow Bwd Byts', 'Subflow Fwd Byts', 'Fwd Header Len', 'Bwd PSH Flags', 'Idle Std', 'Fwd Pkts/s', 'Fwd Act Data Pkts', 'Pkt Len Max', 'Fwd IAT Mean', 'Active Max', 'Active Std', 'Pkt Len Std', 'Pkt Len Var', 'Subflow Fwd Pkts', 'Flow Byts/s', 'Pkt Size Avg', 'Active Mean', 'ACK Flag Cnt', 'Flow IAT Max', 'Active Min', 'Pkt Len Mean', 'Fwd Pkt Len Mean', 'Flow IAT Std', 'Fwd Seg Size Avg', 'PSH Flag Cnt', 'FIN Flag Cnt']


In [86]:
alert2 = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist2:
        alert2.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert2)))

Alert list length:699
